In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import layers, models
import cv2
import os
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping , ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import Input
print('DONE')


DONE


In [2]:
import cv2
import os
import random
from tqdm import tqdm

# Define paths for validation folders
val_violence_path = "/kaggle/input/rwf2000/RWF-2000/val/Fight"
val_nonviolence_path = "/kaggle/input/rwf2000/RWF-2000/val/NonFight"

# Output paths for extracted frames
output_paths = {
    "train_violence": "/kaggle/working/extracted_frames/train/violence",
    "train_nonviolence": "/kaggle/working/extracted_frames/train/nonviolence",
    "val_violence": "/kaggle/working/extracted_frames/val/violence",
    "val_nonviolence": "/kaggle/working/extracted_frames/val/nonviolence"
}

# Create output directories if they don't exist
for path in output_paths.values():
    os.makedirs(path, exist_ok=True)

# Function to extract frames at a rate of 3 frames per second
def extract_frames(video_path, output_folder, target_fps=3):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return
    
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(original_fps / target_fps)
    frame_count, saved_frame_count = 0, 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_interval == 0:
            frame_filename = f"{os.path.splitext(os.path.basename(video_path))[0]}_frame{saved_frame_count}.jpg"
            cv2.imwrite(os.path.join(output_folder, frame_filename), frame)
            saved_frame_count += 1
        frame_count += 1
    cap.release()

# Function to process videos, splitting 10% for validation if needed
def process_videos(video_paths, train_output, val_output, split_for_val=False, target_fps=3):
    for folder in video_paths:
        video_files = [f for f in os.listdir(folder) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
        
        if split_for_val:
            random.shuffle(video_files)
            val_count = max(1, int(len(video_files) * 0.2))  
            val_videos, train_videos = video_files[:val_count], video_files[val_count:]
        else:
            val_videos, train_videos = [], video_files  # All to training if no split
        
        # Extract frames from validation videos
        for video_file in tqdm(val_videos, desc=f"Extracting validation frames from {folder}"):
            extract_frames(os.path.join(folder, video_file), val_output, target_fps=target_fps)

        # Extract frames from training videos
        for video_file in tqdm(train_videos, desc=f"Extracting training frames from {folder}"):
            extract_frames(os.path.join(folder, video_file), train_output, target_fps=target_fps)

# Process violence and nonviolence folders with a 10% split for validation
process_videos(
    ["/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence"],
    output_paths["train_violence"],
    output_paths["val_violence"],
    split_for_val=True
)
process_videos(
    ["/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence"],
    output_paths["train_nonviolence"],
    output_paths["val_nonviolence"],
    split_for_val=True
)

# Process all files from RWF-2000 train folders as training data (no validation split)
process_videos(
    ["/kaggle/input/rwf2000/RWF-2000/train/Fight"],
    output_paths["train_violence"],
    output_paths["val_violence"],
    split_for_val=True
)
process_videos(
    ["/kaggle/input/rwf2000/RWF-2000/train/NonFight"],
    output_paths["train_nonviolence"],
    output_paths["val_nonviolence"],
    split_for_val=True
)


print("Frame extraction completed.")


Extracting validation frames from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence: 100%|██████████| 200/200 [00:32<00:00,  6.14it/s]
Extracting training frames from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence: 100%|██████████| 800/800 [02:32<00:00,  5.25it/s]
Extracting validation frames from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence: 100%|██████████| 200/200 [00:21<00:00,  9.40it/s]
Extracting training frames from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence:  74%|███████▍  | 593/800 [00:57<00:26,  7.81it/s][h264 @ 0x57d002aa7540] mb_type 104 in P slice too large at 98 31
[h264 @ 0x57d002aa7540] error while decoding MB 98 31
Extracting training frames from /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence: 100%|██████████| 800/800 [01:15<00:00, 10.53it/s]
Extracting va

Frame extraction completed.


In [3]:
import os

# Specify the path to the main directory
main_dir = '/kaggle/working/extracted_frames'

# Dictionary to store file counts
file_counts = {}

# Walk through each subdirectory and count files
for root, dirs, files in os.walk(main_dir):
    # Only count files, skip directories
    file_counts[root] = len(files)

# Print the file counts in each folder
for folder, count in file_counts.items():
    print(f"{folder}: {count} files")


/kaggle/working/extracted_frames: 0 files
/kaggle/working/extracted_frames/train: 0 files
/kaggle/working/extracted_frames/train/nonviolence: 22630 files
/kaggle/working/extracted_frames/train/violence: 23781 files
/kaggle/working/extracted_frames/val: 0 files
/kaggle/working/extracted_frames/val/nonviolence: 6156 files
/kaggle/working/extracted_frames/val/violence: 5546 files


In [4]:
# Paths to augmented frames for training and validation
train_dir = '/kaggle/working/extracted_frames/train'
val_dir = '/kaggle/working/extracted_frames/val'
# Image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

# Data augmentation and generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'  
)


val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle= True ,
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False,
)

Found 46411 images belonging to 2 classes.
Found 11702 images belonging to 2 classes.


In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Define the MobileNetV3Small model
input_tensor = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model = MobileNetV3Small(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), include_top=False, weights='imagenet')

base_model.trainable = True 

# Add custom layers on top of MobileNetV3Small
x = base_model(input_tensor)
x = GlobalAveragePooling2D()(x)

x = Dense(1024)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

x = Dense(512)(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)

output = Dense(1, activation='sigmoid')(x)

# Build the model
model = models.Model(inputs=input_tensor, outputs=output)

model.summary()

4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MobilenetV3small (Functional)   │ (None, 7, 7, 576)      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 576)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       590,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_18 (Activation)      │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_19 (Activation)      │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,055,281 (7.84 MB)

 Trainable params: 2,043,169 (7.79 MB)

 Non-trainable params: 12,112 (47.31 KB)

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint 

model.compile(optimizer=Adam(learning_rate=1e-4),  # Lower LR for fine-tuning
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks for adaptive learning rate and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)  
#early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
checkpoint = ModelCheckpoint(
    filepath='best_model.keras',  
    monitor='val_loss',       
    save_best_only=True,     
    mode='min',              
    verbose=1                
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=[checkpoint,reduce_lr]
)

Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1744452199.800784    8108 service.cc:145] XLA service 0x788084005530 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744452199.800840    8108 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


   1/2901 ━━━━━━━━━━━━━━━━━━━━ 53:37:27 67s/step - accuracy: 0.5625 - loss: 0.6417

I0000 00:00:1744452246.251009    8108 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2901/2901 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step - accuracy: 0.7251 - loss: 0.5139
Epoch 1: val_loss improved from inf to 0.62080, saving model to best_model.keras
2901/2901 ━━━━━━━━━━━━━━━━━━━━ 746s 234ms/step - accuracy: 0.7251 - loss: 0.5139 - val_accuracy: 0.6466 - val_loss: 0.6208 - learning_rate: 1.0000e-04
Epoch 2/30
2900/2901 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - accuracy: 0.8777 - loss: 0.2764
Epoch 2: val_loss did not improve from 0.62080
2901/2901 ━━━━━━━━━━━━━━━━━━━━ 641s 220ms/step - accuracy: 0.8777 - loss: 0.2764 - val_accuracy: 0.5635 - val_loss: 1.3486 - learning_rate: 1.0000e-04
Epoch 3/30
2900/2901 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - accuracy: 0.9161 - loss: 0.1991
Epoch 3: val_loss did not improve from 0.62080
2901/2901 ━━━━━━━━━━━━━━━━━━━━ 631s 217ms/step - accuracy: 0.9161 - loss: 0.1991 - val_accuracy: 0.5355 - val_loss: 2.7459 - learning_rate: 1.0000e-04
Epoch 4/30
2901/2901 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.9331 - loss: 0.1589
Epoch 4: val_loss did n

In [7]:
from sklearn.metrics import classification_report
import numpy as np

# Step 1: Evaluate the model using the validation generator
loss, accuracy = model.evaluate(val_generator, verbose=1)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Step 2: Generate predictions
predictions = model.predict(val_generator, verbose=1)          
predicted_classes = (predictions > 0.5).astype(int).reshape(-1)  # Binary classification: Threshold at 0.5

# Step 3: Get true labels from the generator
true_classes = val_generator.classes  # Assuming this provides the true labels for each sample
class_labels = ['Class 0', 'Class 1']  # Replace with actual class names if available

# Step 4: Generate classification report
print("\nClassification Report:\n")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))


732/732 ━━━━━━━━━━━━━━━━━━━━ 29s 40ms/step - accuracy: 0.9114 - loss: 0.5142
Validation Loss: 0.5931624174118042
Validation Accuracy: 0.8960861563682556
732/732 ━━━━━━━━━━━━━━━━━━━━ 37s 46ms/step

Classification Report:

              precision    recall  f1-score   support

     Class 0       0.90      0.91      0.90      6156
     Class 1       0.90      0.88      0.89      5546

    accuracy                           0.90     11702
   macro avg       0.90      0.90      0.90     11702
weighted avg       0.90      0.90      0.90     11702



In [8]:
# Save the model in .keras format
model.save("movileV3_89.keras")


In [9]:
pip install tf2onnx
D

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 9.7 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import tf2onnx
import tensorflow as tf

# Assuming 'model' is your trained Keras or TensorFlow model

# Define the path to save the ONNX model
onnx_model_path = "movileV3_89.onnx"

# Convert the model to ONNX format
spec = (tf.TensorSpec(model.inputs[0].shape, tf.float32, name="input"),)
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=spec)

# Save the ONNX model to the specified path
with open(onnx_model_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print(f"Model saved in ONNX format at {onnx_model_path}")


Model saved in ONNX format at movileV3_89.onnx


In [11]:
from tensorflow import keras
model = keras.models.load_model("/kaggle/working/best_model.keras")


In [12]:
import cv2
import numpy as np

def extract_frames_for_prediction(video_path, target_size=(224, 224), frame_rate=3, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval = int(fps / frame_rate)
    
    count, extracted = 0, 0
    while cap.isOpened() and extracted < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        if count % interval == 0:
            frame = cv2.resize(frame, target_size)
            frame = frame.astype("float32") / 255.0
            frames.append(frame)
            extracted += 1
        count += 1
    cap.release()
    return np.array(frames)


In [14]:
video_path = "/kaggle/input/rwf2000/RWF-2000/val/Fight/1MVS2QPWbHc_0.avi"
frames = extract_frames_for_prediction(video_path)

# Average prediction across all frames
predictions = model.predict(frames)
avg_pred = np.mean(predictions)

label = "Violent" if avg_pred > 0.5 else "Non-Violent"
print(f"Prediction: {label} ({avg_pred:.4f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Prediction: Violent (0.9049)
